In [1]:
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

In [2]:
# Загрузка датасета MNIST с использованием пользовательской функции
# train, validation и test содержат изображения и соответствующие им метки
from my_mnist import load_mnist

train, validation, test = load_mnist()

<class 'urllib.error.URLError'>: <urlopen error unknown url type: https>

In [ ]:
# Извлекаем изображения и метки классов из обучающей выборки

x, y = train[0].reshape(-1, 28, 28), train[1]

x_validation, y_validation = validation[0].reshape(-1, 28, 28), validation[1]

# Тестовая выборка
x_test, y_test = test

In [ ]:
fig, axs = plt.subplots(5, 5, figsize=(20, 20))

x_subset = x[y]

for i in range(5):
    for j in range(5):
        idx = 5 * i + j
        axs[i, j].imshow(x[idx], cmap="gray")
        axs[i, j].axis("off")

In [ ]:
# Реализация собственного класса метода главных компонент (PCA)
class my_PCA():
    def __init__(self, n_components=None):
        self.n_components = n_components
        self.mean = None
        self.components = None
        self.explained_variance = None
    def fit(self, X):
        self.mean = np.mean(X, axis=0)
        X_centered = X - self.mean
        U, s, Vt = np.linalg.svd(X_centered, full_matrices=False)
        self.components = Vt
        self.explained_variance = (s ** 2) / (X.shape[0] - 1)
        max_components = min(X.shape[0], X.shape[1])
        if self.n_components is not None:
            if self.n_components > max_components:
                print(
                    f"Внимание: запрошено {self.n_components} компонент, "
                    f"но доступно только {max_components}"
                )
                self.n_components = max_components

            # Оставляем только первые n_components компонент
            self.components = self.components[:self.n_components]
            self.explained_variance = self.explained_variance[:self.n_components]

        return self

    def transform(self, x):
        # Проверяем, был ли выполнен метод fit
        if self.mean is None or self.components is None:
            raise ValueError("Сначала необходимо вызвать метод fit()")

        x_centered = x - self.mean

        # Проецируем данные на пространство главных компонент
        x_transformed = np.dot(x_centered, self.components.T)
        return x_transformed

    def fit_transform(self, x):
        # Последовательно выполняем fit и transform
        self.fit(x)
        return self.transform(x)
